In [4]:
import requests
import json
import os 
import pandas as pd
import time
import random
import threading
from concurrent.futures import ThreadPoolExecutor
url_image = "https://cdn2.cellphones.com.vn/insecure/rs:fill:358:358/q:90/plain/https://cellphones.com.vn/media/catalog/product/"

In [ ]:
def get_data_product_flash_sale(flash_sale_id = 130, province_id = 5):
    url = "https://api.cellphones.com.vn/graphql-dashboard/graphql/query"
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:137.0) Gecko/20100101 Firefox/137.0',
        'Accept': 'application/json',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate, br, zstd',
        'Content-Type': 'application/json',
        'Origin': 'https://cellphones.com.vn',
        'Connection': 'keep-alive',
        'Referer': 'https://cellphones.com.vn/',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-site',
        'DNT': '1',
        'Sec-GPC': '1',
        'Cookie': 'cps_province_id=5'
    }
    query = """
    query FlashSaleProgram($flash_sale_id: Int!,$province_id: Int!) {
        flash_sale_programs(
            flash_sale_id: $flash_sale_id,
            province_id: $province_id
        ) {
            id
            title
            is_active
            apply_member_type
            apply_for_parent
            started_at
            ended_at
            created_at
            updated_at
            shown_at
            is_show_price
            flash_sale_slots {
                id
                flash_sale_id
                product_id
                max_slot
                sold
                is_active
                flash_sale_prices {
                    id
                    province_id
                    product_id
                    flash_sale_slot_id
                    product_normal_price
                    product_special_price
                    product_flash_sale_price
                    hide_price
                }
                product_name
            product_thumbnail
            product_url
            }
        }
    }
    """
    var = {
        "flash_sale_id":flash_sale_id,
        "province_id": province_id
    }

    payload = json.dumps({
        "query" : query,
        "variables":var
    })

    response = requests.request("POST", url, headers=headers, data=payload)

    return json.loads(response.text)



In [100]:
data = get_data_product_flash_sale()
data = data["data"]
data

{'flash_sale_programs': {'id': 130,
  'title': '11/12',
  'is_active': True,
  'apply_member_type': ['all'],
  'apply_for_parent': True,
  'started_at': '2024-12-11T09:00:00+07:00',
  'ended_at': '2024-12-11T11:00:00+07:00',
  'created_at': '2024-12-09T10:12:52+07:00',
  'updated_at': '2024-12-11T09:49:40+07:00',
  'shown_at': '2024-12-10T11:00:00+07:00',
  'is_show_price': True,
  'flash_sale_slots': [{'id': 2134,
    'flash_sale_id': 130,
    'product_id': 77351,
    'max_slot': 50,
    'sold': 9,
    'is_active': True,
    'flash_sale_prices': [{'id': 153026,
      'province_id': 5,
      'product_id': 81864,
      'flash_sale_slot_id': 2134,
      'product_normal_price': 16990000.0,
      'product_special_price': 15090000.0,
      'product_flash_sale_price': 14890000.0,
      'hide_price': ''},
     {'id': 153027,
      'province_id': 5,
      'product_id': 81808,
      'flash_sale_slot_id': 2134,
      'product_normal_price': 16990000.0,
      'product_special_price': 15090000.0,


In [ ]:

data_flash_sale = data["flash_sale_programs"]

base_path = f"./data_flash_sale/flash_sale_programs/flash_sale_id_{data_flash_sale['id']}"

slots_sale = data_flash_sale["flash_sale_slots"]
for slot in slots_sale: 
    path_dir = os.path.join(base_path,f"sale_slots_{slot['id']}")
    if os.path.isdir(path_dir):
        pass
    else:
        os.makedirs(path_dir, exist_ok = True)
    data_slot= pd.DataFrame(slot)
    columns = ["id","product_id","max_slot","sold","is_active", "product_name","product_url"]

    data_fs = data_slot.loc[:,columns]
    data_fs.to_csv(os.path.join(path_dir, "infor_flash_sale.csv"), index=False)

    data_prices = pd.DataFrame(slot["flash_sale_prices"])
    data_prices.to_csv(os.path.join(path_dir,"prices_sale.csv"),index=False)
    image_path = slot["product_thumbnail"]
    image = requests.get(url_image + image_path)
    if image.status_code == 200:
        with open(os.path.join(path_dir,"image_product.jpg"), "wb") as f:
            f.write(image.content)
    else:
        print("Error down image : ", image.status_code)
    

In [2]:
def get_data_product(categories = None,province_id = 5, stock_from = 1, stock_available_id = None, page = 1, size = 20, sort = None):
  if categories is None:
        categories = ["3"]
  if stock_available_id is None:
        stock_available_id = [46, 152, 4920]
  if sort is None:
        sort = [{"view": "desc"}]
  url = "https://api.cellphones.com.vn/v2/graphql/query"
    
  headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:137.0) Gecko/20100101 Firefox/137.0',
    'Accept': 'application/json',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Content-Type': 'application/json',
    'Origin': 'https://cellphones.com.vn',
    'Connection': 'keep-alive',
    'Referer': 'https://cellphones.com.vn/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'DNT': '1',
    'Sec-GPC': '1',
    'TE': 'trailers',
    'Cookie': 'cps_province_id=5'
  }
  query = """
    query GetProductsByCateId(
      $categories: [String!], 
      $province_id: Int!, 
      $stock_from: Int, 
      $stock_available_id: [Int!], 
      $page: Int, 
      $size: Int, 
      $sort: [Map]
    ) {
      products(
        filter: {
          static: {
            categories: $categories, 
            province_id: $province_id,
            stock: {
              from: $stock_from
            },
            stock_available_id: $stock_available_id
          },
          dynamic: { } 
        },
        page: $page,
        size: $size,
        sort: $sort
      ) {
        general {
          product_id
          name
          sku
          doc_quyen
          manufacturer
          url_key
          url_path
          categories {
            categoryId
            name
            uri
          }
          review {
            total_count
            average_rating
          }
        }
        filterable {
          is_installment
          stock_available_id
          company_stock_id
          filter {
            id
            Label
          }
          is_parent
          price
          prices
          special_price
          promotion_information
          thumbnail
          promotion_pack
          sticker
          flash_sale_types
        }
      }
    }
"""

  var = {
      "categories":categories,
      "province_id": province_id,
      "stock_from": stock_from,
      "stock_available_id": stock_available_id,
      "page": page,
      "size": size,
      "sort": sort
  }
  payload = json.dumps({
    "query": query,
    "variables": var
    }
  )
  try:
    response = requests.request("POST", url=url, headers= headers, data=payload)
    if response.status_code == 429:
        print("Too many requests, sleeping...")
        time.sleep(60)
  except Exception as e:
    print(e)
    time.sleep(10)

  return json.loads(response.text)

In [ ]:
data_products = get_data_product(categories=["30"])

data_products

[{'general': {'product_id': 59258,
   'name': 'iPhone 16 Pro Max 256GB | Chính hãng VN/A',
   'sku': 'iphone-16-pro-max',
   'doc_quyen': '',
   'manufacturer': 'Apple',
   'url_key': 'iphone-16-pro-max',
   'url_path': 'iphone-16-pro-max.html',
   'categories': [{'categoryId': 2, 'name': 'Root', 'uri': 'default-category'},
    {'categoryId': 3, 'name': 'Điện thoại', 'uri': 'mobile'},
    {'categoryId': 132, 'name': 'Apple', 'uri': 'apple'},
    {'categoryId': 1131,
     'name': 'Điện thoại gaming',
     'uri': 'dien-thoai-gaming'},
    {'categoryId': 1601,
     'name': 'Điện thoại pin trâu',
     'uri': 'dien-thoai-pin-trau'},
    {'categoryId': 1602,
     'name': 'Điện thoại dung lượng lớn',
     'uri': 'dien-thoai-dung-luong-lon'},
    {'categoryId': 1603,
     'name': 'Điện thoại cấu hình cao',
     'uri': 'dien-thoai-cau-hinh-cao'},
    {'categoryId': 1605,
     'name': 'Điện thoại chụp ảnh/quay phim đẹp',
     'uri': 'dien-thoai-chup-anh-quay-phim'},
    {'categoryId': 2221, 'nam

In [ ]:
product = data_products[0]
general = product["general"]
print(type(general))
filterable = product["filterable"]

<class 'dict'>


In [64]:

prices = filterable["prices"]
prices["product_id"] = general["product_id"]
prices
# general_data = pd.DataFrame([general])[columns_general]
# categories_id = [cate["categoryId"] for cate in general["categories"]]
# general_data["categories_id"] = [categories_id]
# rate_data = general["review"]
# rate_data["product_id"] = general["product_id"]
# general_data





{'root': {'chiet_khau': 0,
  'discount_id': 0,
  'discount_value': 0,
  'value': 34990000},
 'smem': {'chiet_khau': 152000,
  'discount_id': 0,
  'discount_value': 0,
  'value': 30238000},
 'special': {'chiet_khau': 0,
  'discount_id': 2589,
  'discount_value': 4600000,
  'value': 30390000},
 'svip': {'chiet_khau': 304000,
  'discount_id': 0,
  'discount_value': 0,
  'value': 30086000},
 'product_id': 59258}

In [ ]:
path_dir_products_data = "./data_products/data"
path_dir_products_images = "./data_products/images_product"

columns_general = ["product_id","name","sku","doc_quyen","manufacturer","url_key","url_path"]
columns_filterable = ['is_installment','stock_available_id','company_stock_id','is_parent','price','special_price','flash_sale_types']

data_products = []
categories_data = []
filter_data = []
promotion_data = []
prices_data = []
rate_data = []
attributes_data = []

lock = threading.Lock()

os.makedirs(path_dir_products_data, exist_ok=True)
os.makedirs(path_dir_products_images, exist_ok=True)

def get_data_product_home(category):
        page = 1
        size = 1000

        print(f"Start crawl with page {page} and category {category}")
        try:
            response= get_data_product(page = page ,size=size, categories= [category])
            if not response:
                print("Page have no data")
                return
            if "products" not in response["data"] or "data" not in response:
                print(f"Page have no data and unexpected structure : {response}")
                return
            data_products_source = list(response["data"]["products"])
        except Exception as e:
            print(f"Failed to get in page {page} and category {category} because : {e}")
            return
        
        for product in data_products_source:
            general = product.get("general", {})
            filterable = product.get("filterable", {})
            product_id = general.get("product_id", "unknown")

            path_image = os.path.join(path_dir_products_images, f"image_product_{product_id}.png")

            attribute = general.get("atrributes",{})

            product_data_general = {key: general.get(key, "") for key in columns_general}
            product_data_filterable = {key: filterable.get(key, "") for key in columns_filterable}
            merge_data = {**product_data_general, **product_data_filterable}
            merge_data["path_image"] = path_image
            


            categories = general.get("categories", [])
            review = general.get("review")
            filter_section = filterable.get("filter")
            prices = filterable.get("prices")
            promotion = filterable.get("promotion_pack", {}).get("km_chung")

            with lock:
                # if attribute:
                #     attribute["product_id"] = product_id
                #     attributes_data.append(attribute)

                data_products.append(merge_data)
                
                for category in categories:
                    category["product_id"] = product_id
                categories_data.extend(categories)

                if review:
                    review["product_id"] = product_id
                    rate_data.append(review)

                if isinstance(filter_section, list):
                    for f in filter_section:
                        f["product_id"] = product_id
                    filter_data.extend(filter_section)
                elif isinstance(filter_section, dict):
                    filter_section["product_id"] = product_id
                    filter_data.append(filter_section)

                if prices:
                    prices["product_id"] = product_id
                    prices_data.append(prices)

                if promotion:
                    promotion["product_id"] = product_id
                    promotion_data.append(promotion)


            thumbnail = filterable.get("thumbnail")
            if thumbnail:
                try:
                    image = requests.get(url_image + thumbnail)
                    with open(path_image, "wb") as img:
                        img.write(image.content)
                except:
                    print(f"Failed to download image for product {product_id}")

                    
if __name__ == "__main__":
    categories = ["30"]
    print("Begin Crawl")

    with ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(get_data_product_home, categories)
    
    print("All crawl page done")


    pd.DataFrame(data_products).to_csv(os.path.join(path_dir_products_data,"data_products.csv"), index=False)
    pd.DataFrame(categories_data).to_csv(os.path.join(path_dir_products_data,"data_categories.csv"), index=False)

    with open(os.path.join(path_dir_products_data,"data_filters.json"), "w", encoding="utf-8") as f:
        json.dump(filter_data, f, indent=4, ensure_ascii=False)

    with open(os.path.join(path_dir_products_data,"data_rates.json"), "w", encoding="utf-8") as f:
        json.dump(rate_data, f, indent=4, ensure_ascii=False)

    with open(os.path.join(path_dir_products_data,"data_promotions.json"), "w", encoding="utf-8") as f:
        json.dump(promotion_data, f, indent=4, ensure_ascii=False)

    with open(os.path.join(path_dir_products_data,"data_prices.json"), "w", encoding="utf-8") as f:
        json.dump(prices_data, f, indent=4, ensure_ascii=False)

    print("Data save ok")


Begin Crawl
Start crawl with page 1 and category 30


KeyboardInterrupt: 